## Lab 4 Familias de Malware

#### Brandon Ronaldo Sicay Cumes - 21757

### **Parte 1**

#### Creación del dataset

Analisis estatico de Malware


In [1]:
%pip install pefile


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import pefile

In [4]:
import os
import pandas as pd
import pefile

# Función para extraer características de un archivo PE
def extract_pe_features(file_path):
    pe_features = {}

    # Abrir el archivo PE
    try:
        pe = pefile.PE(file_path)
    except pefile.PEFormatError:
        return None

    # PE Header
    pe_features['Machine'] = pe.FILE_HEADER.Machine
    pe_features['NumberOfSections'] = pe.FILE_HEADER.NumberOfSections
    pe_features['TimeDateStamp'] = pe.FILE_HEADER.TimeDateStamp
    pe_features['PointerToSymbolTable'] = pe.FILE_HEADER.PointerToSymbolTable
    # Agregar más características del PE header según sea necesario

    # Secciones
    section_names = [section.Name.decode().strip('\x00') for section in pe.sections]
    pe_features['SectionNames'] = ';'.join(section_names)

    # Imports
    if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
        imports = [entry.dll.decode() for entry in pe.DIRECTORY_ENTRY_IMPORT]
        pe_features['Imports'] = ';'.join(imports)
    else:
        pe_features['Imports'] = None

    # Otros análisis estáticos que desees realizar

    return pe_features

# Directorio que contiene los archivos de malware
malware_dir = './MALWR'

# Lista para almacenar las características de todos los archivos
data = []

# Recorrer los archivos en el directorio de malware
for filename in os.listdir(malware_dir):
    file_path = os.path.join(malware_dir, filename)
    if os.path.isfile(file_path):
        # Extraer características del archivo PE
        pe_features = extract_pe_features(file_path)
        if pe_features is not None:
            pe_features['Filename'] = filename
            data.append(pe_features)

# Convertir la lista de diccionarios a un DataFrame de Pandas
df = pd.DataFrame(data)

# Guardar el DataFrame como un archivo CSV
df.to_csv('malware_dataset.csv', index=False)

#### Exploración y pre-procesamiento de datos

In [5]:
# Leer el dataset desde el archivo CSV
df = pd.read_csv('malware_dataset.csv')

# Mostrar las primeras filas del dataset
print("----- Primeras filas del dataset:")
print(df.head())

# Resumen estadístico de las características numéricas
print("\n----- Resumen estadístico - características numéricas:")
print(df.describe())

# Tipos de datos de cada columna
print(df.dtypes)

# Número de filas y columnas en el dataset
print(df.shape)

# Mostrar algunas estadísticas adicionales
print(df['NumberOfSections'].max())

print(df['NumberOfSections'].min())

print(df['Machine'].nunique())

# Explorar las primeras secciones de cada muestra
print("\nPrimeras secciones de cada muestra:")
for index, row in df.iterrows():
    print(f"\nMuestra: {row['Filename']}")
    print(row['SectionNames'].split(';')[0])  # Mostrar solo la primera sección para simplificar la visualización

# Explorar las primeras funciones importadas de cada muestra
print("\nPrimeras funciones importadas de cada muestra:")
for index, row in df.iterrows():
    print(f"\nMuestra: {row['Filename']}")
    if row['Imports']:
        print(row['Imports'].split(';')[:5])  # Mostrar solo las primeras 5 funciones importadas para simplificar la visualización
    else:
        print("No hay información de importaciones para esta muestra.")

----- Primeras filas del dataset:
   Machine  NumberOfSections  TimeDateStamp  PointerToSymbolTable  \
0      332                 3     1242321160                     0   
1      332                 3     1242321160                     0   
2      332                 3     1311923431                     0   
3      332                 3     1242321160                     0   
4      332                 3     1242321160                     0   

         SectionNames                                            Imports  \
0     UPX0;UPX1;.rsrc  KERNEL32.DLL;MSVCRT.dll;SHELL32.dll;USER32.dll...   
1     UPX0;UPX1;.rsrc  KERNEL32.DLL;MSVCRT.dll;SHELL32.dll;USER32.dll...   
2  .text;.rdata;.data  KERNEL32.dll;USER32.dll;ADVAPI32.dll;SHELL32.d...   
3     UPX0;UPX1;.rsrc  KERNEL32.DLL;MSVCRT.dll;SHELL32.dll;USER32.dll...   
4     UPX0;UPX1;.rsrc  KERNEL32.DLL;MSVCRT.dll;SHELL32.dll;USER32.dll...   

                                Filename  
0   JH78C0A33A1B472A8C16123FD696A5CE5EBB  
1   NBV_

In [ ]:
# Columnas categóricas y numéricas
categorical_cols = ['SectionNames', 'Imports']
numeric_cols = ['Machine', 'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable']

# Preprocesamiento de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Escalado de características numéricas
        ('cat', OneHotEncoder(), categorical_cols)  # Codificación one-hot de características categóricas
    ])

# Definir el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Aplicar el preprocesamiento al DataFrame
df_preprocessed = pipeline.fit_transform(df)

# Convertir el resultado a un nuevo DataFrame
df_preprocessed = pd.DataFrame(df_preprocessed)

# Agregar el nombre de las columnas al DataFrame preprocesado
column_names = numeric_cols + list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out())
df_preprocessed.columns = column_names

# Mostrar el DataFrame preprocesado
print(df_preprocessed.head())

### **Parte 2**


#### Implementación del modelo

#### Análisis de similitud

### **Conclusiones**
